## Máster en Big Data y Data Science

### Metodologías de gestión y diseño de proyectos de big data

#### AP2 - Descubrimiento de grupos en los datos

---

En esta libreta se generan los modelos de clusterización sobre el dataset final del escenario para su posterior evaluación y análisis. Se utilizará como herramienta de soporte a mlflow para el registro completo de la experimentación. 

---

In [2]:
# Importación de librerías

import pandas as pd
# Se importa mlflow para registro de la experimentación 
import mlflow
# Se importan los métodos a utilizar para clusterizar
from sklearn.cluster import DBSCAN, KMeans, AgglomerativeClustering, MeanShift
# Se importan las métricas a utilizar para evaluar el proceso
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score


/Users/salvarodriguezabad/Desktop/13MBID-Metodologias-Big-Data/.env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


----

##### Lectura del dataset

In [5]:
df = pd.read_csv("../../../data/final/datos_clusterizacion.csv", sep=";")
df.head(5)

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,ingresos,pct_ingreso,tasa_interes,estado_credito,antiguedad_cliente,...,estado_cliente_ACTIVO,estado_cliente_PASIVO,genero_F,genero_M,nivel_educativo_DESCONOCIDO,nivel_educativo_POSGRADO_COMPLETO,nivel_educativo_POSGRADO_INCOMPLETO,nivel_educativo_SECUNDARIO_COMPLETO,nivel_educativo_UNIVERSITARIO_COMPLETO,nivel_educativo_UNIVERSITARIO_INCOMPLETO
0,713061558.0,22,35000,3,123.0,59000,0.59,16.02,1,36.0,...,True,False,False,True,False,False,False,False,True,False
1,768805383.0,21,1000,2,5.0,9600,0.10,11.14,0,39.0,...,True,False,False,True,False,False,False,True,False,False
2,713982108.0,23,35000,2,4.0,65500,0.53,15.23,1,36.0,...,True,False,False,True,False,False,False,False,True,False
3,710821833.0,24,35000,4,8.0,54400,0.55,14.27,1,54.0,...,True,False,False,True,True,False,False,False,False,False
4,769911858.0,21,2500,2,2.0,9900,0.25,7.14,1,34.0,...,True,False,True,False,False,False,False,True,False,False


----

##### Configuración de la experimentación

In [3]:
# Se genera el experimento en mlflow
exp_name = 'Experimentación clustering'
exp_id = mlflow.create_experiment(name=exp_name)

In [4]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id, run_name="Kmeans - K=2"):
    # Selección y configuración de la técnica
    modelo_clusters = KMeans(n_clusters=2)
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # Se registra el parámetro K
    mlflow.log_param('Valor K', 2)

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    mlflow.sklearn.log_model(trained_model, "Kmeans_K2")

    # Se finaliza el registro
    mlflow.end_run()

Para contabilizar los elementos por cluster se utilizan las labels generadas

In [5]:
# Se preserva el conjunto de datos original para evitar problemas
df_kmeans_k2 = df.copy()
# Se agregan las labels generadas
df_kmeans_k2['cluster'] = cluster_labels
df_kmeans_k2['cluster'].value_counts()

cluster
0    6053
1    2832
Name: count, dtype: int64

Se pasa a probar otro método (clustering jerárquico)

In [6]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id, run_name="AGClustering - K=3"):
    # Selección y configuración de la técnica
    modelo_clusters = AgglomerativeClustering(n_clusters=3)
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # Se registra el parámetro K
    mlflow.log_param('Valor K', 3)

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    mlflow.sklearn.log_model(trained_model, "AGC_K3")

    # Se finaliza el registro
    mlflow.end_run()

2024/04/18 13:24:10 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


Para contabilizar los elementos por cluster se utilizan las labels generadas

In [7]:
# Se preserva el conjunto de datos original para evitar problemas
df_agc_k3 = df.copy()
# Se agregan las labels generadas
df_agc_k3['cluster'] = cluster_labels
df_agc_k3['cluster'].value_counts()

cluster
1    5966
0    2073
2     846
Name: count, dtype: int64

Finalmente, se prueba un método que no tiene parámetros

In [8]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id, run_name="MeanShift"):
    # Selección y configuración de la técnica
    modelo_clusters = MeanShift()
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # No hay parámetros por registrar

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    mlflow.sklearn.log_model(trained_model, "MS")

    # Se finaliza el registro
    mlflow.end_run()

Para contabilizar los elementos por cluster se utilizan las labels generadas

In [9]:
# Se preserva el conjunto de datos original para evitar problemas
df_ms = df.copy()
# Se agregan las labels generadas
df_ms['cluster'] = cluster_labels
df_ms['cluster'].value_counts()

cluster
0    6021
1    1944
2     920
Name: count, dtype: int64

----

Prueba Nro. 2

- Kmeans


In [10]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id, run_name="Kmeans - K=3"):
    # Selección y configuración de la técnica
    modelo_clusters = KMeans(n_clusters=3)
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # Se registra el parámetro K
    mlflow.log_param('Valor K', 3)

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    mlflow.sklearn.log_model(trained_model, "Kmeans_K3")

    # Se finaliza el registro
    mlflow.end_run()

In [11]:
# Se preserva el conjunto de datos original para evitar problemas
df_kmeans_k3 = df.copy()
# Se agregan las labels generadas
df_kmeans_k3['cluster'] = cluster_labels
df_kmeans_k3['cluster'].value_counts()

cluster
0    6018
1    1919
2     948
Name: count, dtype: int64

In [17]:
df_kmeans_k3.to_csv("../../../data/final/resultados_kmeans_k2.csv", sep=";", index=False)

- Clustering Jerárquico

In [12]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id, run_name="AGClustering - K=2"):
    # Selección y configuración de la técnica
    modelo_clusters = AgglomerativeClustering(n_clusters=2)
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # Se registra el parámetro K
    mlflow.log_param('Valor K', 2)

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    mlflow.sklearn.log_model(trained_model, "AGC_K2")

    # Se finaliza el registro
    mlflow.end_run()

2024/04/18 13:54:11 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


In [13]:
# Se preserva el conjunto de datos original para evitar problemas
df_agc_k2 = df.copy()
# Se agregan las labels generadas
df_agc_k2['cluster'] = cluster_labels
df_agc_k2['cluster'].value_counts()

cluster
1    5966
0    2919
Name: count, dtype: int64

----

Experimentación #2

In [3]:
# Se genera el experimento en mlflow
exp_name_2 = 'Experimentación clustering #2'
exp_id_2 = mlflow.create_experiment(name=exp_name_2)

In [6]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id_2, run_name="AGClustering - K=4"):
    # Selección y configuración de la técnica
    modelo_clusters = AgglomerativeClustering(n_clusters=4)
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # Se registra el parámetro K
    mlflow.log_param('Valor K', 4)

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    mlflow.sklearn.log_model(trained_model, "AGC_K4")

    # Se finaliza el registro
    mlflow.end_run()

2024/04/23 17:58:44 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
